In [3]:
!pip install guizero

     ---------------------------------------- 0.0/44.5 kB ? eta -:--:--
     ----------------- -------------------- 20.5/44.5 kB 320.0 kB/s eta 0:00:01
     -------------------------------------- 44.5/44.5 kB 439.2 kB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
from guizero import *
import PIL

In [5]:
!pip install pyarrow

In [6]:
try:
    df=pd.read_feather("miri_raw_data.feather")
except Exception as E:
    print(E)
    df=pd.read_feather("miri_raw_data.backup.feather")

In [7]:
df.head()

,id_reporte,url,year,month,filename,ext,date,text,y,y2,yf
0,959,https://www.cenapred.unam.mx/popo/2000/abr/p04...,2000,abr,p0423001,jpg,"11:00 (16:00 GMT, Abril 23)","\n<b>11:00 (16:00 GMT, Abril 23):</b>\n\n<br/>...",1,1,
1,959,https://www.cenapred.unam.mx/popo/2000/abr/p04...,2000,abr,p0423002,jpg,"11:00 (16:00 GMT, Abril 23)","\n<b>11:00 (16:00 GMT, Abril 23):</b>\n\n<br/>...",1,1,
2,959,https://www.cenapred.unam.mx/popo/2000/abr/p04...,2000,abr,p0423003,jpg,"11:00 (16:00 GMT, Abril 23)","\n<b>11:00 (16:00 GMT, Abril 23):</b>\n\n<br/>...",3,3,
3,960,https://www.cenapred.unam.mx/popo/2000/abr/p04...,2000,abr,p0424001,jpg,"11:00 (16:00 GMT, Abril 24)","\n<b>11:00 (16:00 GMT, Abril 24):</b>\n\n<br/>...",3,3,
4,960,https://www.cenapred.unam.mx/popo/2000/abr/p04...,2000,abr,p0424002,jpg,"11:00 (16:00 GMT, Abril 24)","\n<b>11:00 (16:00 GMT, Abril 24):</b>\n\n<br/>...",3,3,


In [8]:


if "y" not in list(df.columns):
    df["y"]=[""]*df.shape[0]
    df.to_feather("miri_raw_data.feather")
    
if "y2" not in list(df.columns):
    df["y2"]=[""]*df.shape[0]
    df.to_feather("miri_raw_data.feather")
    
if "yf" not in list(df.columns):
    df["yf"]=[""]*df.shape[0]
    df.to_feather("miri_raw_data.feather")

In [9]:
df[df.isnull().any(1)]

TypeError: DataFrame.any() takes 1 positional argument but 2 were given

In [10]:
def path_of(i:int)->str:
    y,m,f,e=df.iloc[i][["year","month","filename","ext"]]
    return f"{y}/{m}/{f}.{e}"
print(path_of(3))

2000/abr/p0424001.jpg


In [11]:
current_image_idx=None
def get_next_image():
    global current_image_idx
    current_image_idx=df.y2.eq("").idxmax()
    return path_of(current_image_idx)


In [12]:
def get_size(path:str) -> tuple[int, int]:
    im = PIL.Image.open(path)
    return im.size

In [10]:
path=get_next_image()
w,h=get_size(path)
app = App(
    height=h+25*5,
    width=w, 
    bg="white"
)


pic = Picture(app, image=path)
TL1 = Text(app, text="[1]=No Activo")
TL2 = Text(app, text="[2]=Explosión")
TL3 = Text(app, text="[3]=Fumarola")
TL4 = Text(app, text="[4]=No sé")


Input= TextBox(app, width="fill", multiline=False)
#Input.bg="black"

Barrier=False
def textupdate():
    global Barrier
    if Barrier:
        return
    value=Input.value.strip()
    Input.value=value
    if len(value)==0 or value[-1]!=".":
        return
    Barrier=True
    
    filtered=[] #filter out the numbers exclusively
    for c in value[:-1]:
        try:
            int(c)
        except:
            continue
        filtered.append(c)
    
    #print("prev_y",df.iloc[current_image_idx,-2])
    df.to_feather("miri_raw_data.backup.feather")
    df.iloc[current_image_idx,-2]=" ".join(filtered)
    df.to_feather("miri_raw_data.feather")
        
    #Load another image and reset the input
    pic.image=get_next_image()

    app.title=f'{df[df["y2"]!=""].shape[0]/df.shape[0]*100:.03f}% complete.'
    Input.value=""
    Barrier=False
Input.update_command(textupdate)
Input.focus()

    
app.display()


In [13]:
# df[df["y"]!=""].shape[0]/df.shape[0]*100

In [14]:
df[df["y2"]!=""].shape[0]/df.shape[0]*100

9.805335255948089

In [15]:
df[df["yf"]!=""].shape[0]/df.shape[0]*100

0.0

In [16]:
df.head(10)

,id_reporte,url,year,month,filename,ext,date,text,y,y2,yf
0,959,https://www.cenapred.unam.mx/popo/2000/abr/p04...,2000,abr,p0423001,jpg,"11:00 (16:00 GMT, Abril 23)","\n<b>11:00 (16:00 GMT, Abril 23):</b>\n\n<br/>...",1,1,
1,959,https://www.cenapred.unam.mx/popo/2000/abr/p04...,2000,abr,p0423002,jpg,"11:00 (16:00 GMT, Abril 23)","\n<b>11:00 (16:00 GMT, Abril 23):</b>\n\n<br/>...",1,1,
2,959,https://www.cenapred.unam.mx/popo/2000/abr/p04...,2000,abr,p0423003,jpg,"11:00 (16:00 GMT, Abril 23)","\n<b>11:00 (16:00 GMT, Abril 23):</b>\n\n<br/>...",3,3,
3,960,https://www.cenapred.unam.mx/popo/2000/abr/p04...,2000,abr,p0424001,jpg,"11:00 (16:00 GMT, Abril 24)","\n<b>11:00 (16:00 GMT, Abril 24):</b>\n\n<br/>...",3,3,
4,960,https://www.cenapred.unam.mx/popo/2000/abr/p04...,2000,abr,p0424002,jpg,"11:00 (16:00 GMT, Abril 24)","\n<b>11:00 (16:00 GMT, Abril 24):</b>\n\n<br/>...",3,3,
5,961,https://www.cenapred.unam.mx/popo/2000/abr/p04...,2000,abr,p0425001,jpg,"11:00 (16:00 GMT, Abril 25)","\n<b>11:00 (16:00 GMT, Abril 25):</b>\n\n<br/>...",3,3,
6,961,https://www.cenapred.unam.mx/popo/2000/abr/p04...,2000,abr,p0425002,jpg,"11:00 (16:00 GMT, Abril 25)","\n<b>11:00 (16:00 GMT, Abril 25):</b>\n\n<br/>...",3,3,
7,962,https://www.cenapred.unam.mx/popo/2000/abr/p04...,2000,abr,p0427001,jpg,"11:00 (16:00 GMT, Abril 27)","\n<b>11:00 (16:00 GMT, Abril 27):</b>\n\n<br/>...",3,1,
8,962,https://www.cenapred.unam.mx/popo/2000/abr/p04...,2000,abr,p0427002,jpg,"11:00 (16:00 GMT, Abril 27)","\n<b>11:00 (16:00 GMT, Abril 27):</b>\n\n<br/>...",4,4,
9,963,https://www.cenapred.unam.mx/popo/2000/abr/p04...,2000,abr,p0428001,jpg,"11:00 (16:00 GMT, Abril 28)","\n<b>11:00 (16:00 GMT, Abril 28):</b>\n\n<br/>...",4,1,


In [17]:
df[df["text"].str.find("explosion")>0].iloc[0].text

'\n<br/>\n\n<b> 9:50 Diciembre 14 (15:50 GMT). </b>\nA las  05:44 del día de hoy se registró un incremento en la actividad del volcán. Lo anterior derivó, minutos más tarde, en la emisión de una columna de ceniza, vapor y gases que alcanzó una altura cercana a los 4 km por encima del cráter. Dicha emisión se ha mantenido desde entonces, con algunas fluctuaciones. Al momento de este reporte se observa una tendencia a disminuir.\n<br/>\nLos vientos dominantes dirigieron la pluma hacia el Oeste, sector donde ya se ha reportado la caída de ceniza (Ozumba, Nexapa y Amecameca).\n<br/>\nAsimismo, se observó, a lo largo de la noche, la emisión esporádica de fragmentos incandescentes en volúmenes reducidos como consecuencia de explosiones de bajo nivel en la base del cráter. Dichos fragmentos han caído en las laderas del volcán, particularmente en las partes altas.\n<br/>\nEs probable que en las próximas horas o días se presenten eventos similares.\n<br/>\n\n<br/>\nSe recomienda no acercarse a 